In [1]:
# 03_modelado.ipynb
import pandas as pd
import numpy as np
import joblib
import json

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score



In [3]:
df = pd.read_csv("datos_credito_simulados_listos.csv")

X = df.drop(columns=['riesgo_real'])
y = df['riesgo_real']

print("Dimensión del dataset:", X.shape)


Dimensión del dataset: (1000, 28)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.30,
    random_state=42,
    stratify=y
)


In [5]:
modelos = {
    "Logistica": LogisticRegression(
        max_iter=1000,
        class_weight='balanced'
    ),
    "RandomForest": RandomForestClassifier(
        n_estimators=300,
        random_state=42,
        class_weight='balanced'
    )
}


In [6]:
resultados = {}
mejor_modelo = None
mejor_auc = 0
nombre_ganador = None

for nombre, modelo in modelos.items():

    print(f"\nEntrenando modelo: {nombre}")

    # ---- Escalado solo para regresión logística ----
    usa_scaler = nombre == "Logistica"
    scaler = None

    if usa_scaler:
        scaler = StandardScaler()
        X_train_proc = scaler.fit_transform(X_train)
        X_test_proc = scaler.transform(X_test)
    else:
        X_train_proc = X_train
        X_test_proc = X_test

    # ---- Entrenamiento ----
    modelo.fit(X_train_proc, y_train)

    # ---- Predicciones ----
    y_pred = modelo.predict(X_test_proc)
    y_prob = modelo.predict_proba(X_test_proc)[:, 1]

    # ---- Métricas ----
    auc = roc_auc_score(y_test, y_prob)
    print(classification_report(y_test, y_pred))
    print(f"AUC: {auc:.4f}")

    resultados[nombre] = auc

    # ---- Selección del mejor modelo ----
    if auc > mejor_auc:
        mejor_auc = auc
        mejor_modelo = modelo
        nombre_ganador = nombre
        mejor_scaler = scaler



Entrenando modelo: Logistica
              precision    recall  f1-score   support

           0       0.88      0.78      0.83       222
           1       0.53      0.71      0.61        78

    accuracy                           0.76       300
   macro avg       0.71      0.74      0.72       300
weighted avg       0.79      0.76      0.77       300

AUC: 0.8132

Entrenando modelo: RandomForest
              precision    recall  f1-score   support

           0       0.86      1.00      0.92       222
           1       0.98      0.54      0.69        78

    accuracy                           0.88       300
   macro avg       0.92      0.77      0.81       300
weighted avg       0.89      0.88      0.86       300

AUC: 0.8075


In [7]:
print("\n===================================")
print(f"✅ Mejor modelo: {nombre_ganador}")
print(f"✅ AUC obtenido: {mejor_auc:.4f}")
print("===================================")



✅ Mejor modelo: Logistica
✅ AUC obtenido: 0.8132


In [8]:
# Ruta base para la app Django
BASE_PATH = "../web_app/credit_risk/ml_models/"

# -------------------------------
# Guardar modelo
# -------------------------------
ruta_modelo = BASE_PATH + "modelo_riesgo.pkl"
joblib.dump(mejor_modelo, ruta_modelo)

# -------------------------------
# Guardar scaler SOLO si aplica
# -------------------------------
if nombre_ganador == "Logistica":
    ruta_scaler = BASE_PATH + "scaler.pkl"
    joblib.dump(mejor_scaler, ruta_scaler)
else:
    ruta_scaler = None

# -------------------------------
# Guardar orden de features
# -------------------------------
ruta_features = BASE_PATH + "features.json"
with open(ruta_features, "w") as f:
    json.dump(list(X.columns), f)

print("Archivos guardados para integración en Django:")
print("Modelo:", ruta_modelo)
if ruta_scaler:
    print("Scaler:", ruta_scaler)
print("Features:", ruta_features)


Archivos guardados para integración en Django:
Modelo: ../web_app/credit_risk/ml_models/modelo_riesgo.pkl
Scaler: ../web_app/credit_risk/ml_models/scaler.pkl
Features: ../web_app/credit_risk/ml_models/features.json


In [ ]:
if nombre_ganador == "RandomForest":
    importancias = pd.Series(
        mejor_modelo.feature_importances_,
        index=X.columns
    ).sort_values(ascending=False)

    importancias.head(10)
